In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_excel('data/civil.xlsx')

In [3]:
df = df.dropna()
df.isnull().sum()

Sl.no              0
Description        0
Case               0
People affected    0
Time period        0
Compensation       0
dtype: int64

In [4]:
df.head()

,Sl.no,Description,Case,People affected,Time period,Compensation
0,1,Disciplinary inquiry against an employee of th...,Employment dispute,4,14.0,0
1,2,Dispute over the termination of agreements for...,Railway dispute,12,23.0,0
2,3,Dispute regarding the validity of a Town Impr...,Civil Appeal,4,25.0,5543000
3,4,"Dispute revolves around the partition of land,...",Civil Appeal,2,21.0,4432675
4,5,Matters related to the classification of proce...,Excise Duty Appeal,6,27.0,0


In [5]:
weights = {
    'People affected': 0.33,
    'Compensation': 0.33,
    'Time period': 0.33,
}

# Convert columns to numeric (excluding non-numeric columns)
numeric_columns = ['People affected', 'Compensation', 'Time period']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Replace any remaining non-numeric values with NaN
df[numeric_columns] = df[numeric_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))

# Normalize each column (including the scaled columns) by dividing by its maximum value
for column in numeric_columns:
    df[column] = df[column] / df[column].max()

# Calculate the "Priority Score" for each row
df['Priority Score'] = df.apply(lambda row: sum(row[column] * weights[column] for column in weights), axis=1)

# Rescale "Priority Score" to the range of 0 to 10
min_score = df['Priority Score'].min()
max_score = df['Priority Score'].max()
df['Priority Score'] = 0 + (df['Priority Score'] - min_score) * (10 - 0) / (max_score - min_score)

# Print the DataFrame with the new "Priority Score" column
print(df)


    Sl.no                                        Description  \
0       1  Disciplinary inquiry against an employee of th...   
1       2  Dispute over the termination of agreements for...   
2       3   Dispute regarding the validity of a Town Impr...   
3       4  Dispute revolves around the partition of land,...   
4       5  Matters related to the classification of proce...   
..    ...                                                ...   
95     96  Victim filed Writ Petition seeking to prevent ...   
96     97  The appellant's application for arrears was in...   
97     98  The case pertains to a dispute over the calcul...   
98     99  Case primarily focuses on the eligibility crit...   
99    100  Dismissal of Sub-Inspector Shardul Singh by th...   

                            Case  People affected  Time period  Compensation  \
0             Employment dispute     1.313629e-06     0.259259      0.000000   
1                Railway dispute     3.940887e-06     0.425926      0.0

In [6]:
df.head()

,Sl.no,Description,Case,People affected,Time period,Compensation,Priority Score
0,1,Disciplinary inquiry against an employee of th...,Employment dispute,1.313629e-06,0.259259,0.000000,1.445086
1,2,Dispute over the termination of agreements for...,Railway dispute,3.940887e-06,0.425926,0.000000,2.485566
2,3,Dispute regarding the validity of a Town Impr...,Civil Appeal,1.313629e-06,0.462963,0.093949,3.303268
3,4,"Dispute revolves around the partition of land,...",Civil Appeal,6.568144e-07,0.388889,0.075130,2.723352
4,5,Matters related to the classification of proce...,Excise Duty Appeal,1.970443e-06,0.500000,0.000000,2.947982


In [7]:
X = df.drop(['Time period','Description'],axis=1)

In [8]:
X.head()

,Sl.no,Case,People affected,Compensation,Priority Score
0,1,Employment dispute,1.313629e-06,0.000000,1.445086
1,2,Railway dispute,3.940887e-06,0.000000,2.485566
2,3,Civil Appeal,1.313629e-06,0.093949,3.303268
3,4,Civil Appeal,6.568144e-07,0.075130,2.723352
4,5,Excise Duty Appeal,1.970443e-06,0.000000,2.947982


In [9]:
Y = df['Time period']

In [10]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns


numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
# the below code is responsible for transforming the columns
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [11]:
X = preprocessor.fit_transform(X)


In [12]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.1,random_state=42)
X_train.shape, X_test.shape

((90, 64), (10, 64))

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.2567
- Mean Absolute Error: 0.2233
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2625
- Mean Absolute Error: 0.2306
- R2 Score: -0.0666


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.0059
- Mean Absolute Error: 0.0045
- R2 Score: 0.9995
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.0300
- Mean Absolute Error: 0.0132
- R2 Score: 0.9861


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.0680
- Mean Absolute Error: 0.0495
- R2 Score: 0.9298
----------------------

In [15]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,1.000000
2,Ridge,0.986077
3,K-Neighbors Regressor,0.823019
7,CatBoosting Regressor,0.445699
1,Lasso,-0.066571
5,Random Forest Regressor,-0.089990
8,AdaBoost Regressor,-0.136847
6,XGBRegressor,-0.163517
4,Decision Tree,-0.353806
